In [1]:
'''
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
'''

'\n# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" \n# You can also write temporary files to /kaggle/temp/, but they won\'t be saved outside of the current session\n'

In [2]:
CLONE_URL = f"https://github.com/LucaBrembilla/RecSys_Course_AT_PoliMi.git"
get_ipython().system(f"git clone {CLONE_URL}")

import sys
sys.path.append("RecSys_Course_AT_PoliMi")

fatal: destination path 'RecSys_Course_AT_PoliMi' already exists and is not an empty directory.


In [1]:
import sys
sys.path.append("RecSys_Course_AT_PoliMi")

In [2]:
!pip install "numpy<1.24.0"

In [3]:
import numpy

In [4]:
print(numpy.__version__)

1.23.5


In [5]:
import time
import pandas as pd
import numpy as np
import scipy.sparse as sps
import random as rnd

from scipy.sparse import *

In [6]:
urm_path = '/kaggle/input/recsyscourse/data_train.csv'
urm_all_df = pd.read_csv(filepath_or_buffer=urm_path,
                                sep=",",
                                header=0,
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

urm_all_df.columns = ["UserID", "ItemID", "Interaction"]


In [7]:
URM_all = sps.coo_matrix((urm_all_df["Interaction"].values,
                          (urm_all_df["UserID"].values, urm_all_df["ItemID"].values)))

In [8]:
del urm_all_df

### Step 1: Split the data and create the evaluator objects

In [9]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [10]:
from Evaluation.Evaluator import EvaluatorHoldout

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2917 (22.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2559 (19.6%) Users that have less than 1 test interactions


### Step 2: Define hyperparameter set for the desired model, in this case rp3beta

In [13]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(200,300),
    "l2_norm": Real(5, 200)
}

### Step 3: Create SearchBayesianSkopt object, providing the desired recommender class and evaluator objects

In [14]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = EASE_R_Recommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation,
                                         evaluator_test=evaluator_test)

### Step 4: Provide the data needed to create an instance of the model, one trained only on URM_train, the other on URM_train_validation

In [15]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

In [16]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

### Step 5: Create a result folder and select the number of cases (50 with 30% random is a good number)

In [17]:
import os

output_folder_path = "result_experiments/easer"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 6  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

### Step 5: Run!

In [18]:
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 266, 'l2_norm': 23.20562723656721}
EASE_R_Recommender: URM Detected 840 ( 6.4%) users with no interactions.
EASE_R_Recommender: URM Detected 461 ( 2.1%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 8.31 min
EvaluatorHoldout: Processed 10036 (100.0%) in 14.34 sec. Users per second: 700
SearchBayesianSkopt: New best config found. Config 0: {'topK': 266, 'l2_norm': 23.20562723656721} - results: PRECISION: 0.0602232, PRECISION_RECALL_MIN_DEN: 0.1192921, RECALL: 0.1069236, MAP: 0.0276196, MAP_MIN_DEN: 0.0552709, MRR: 0.1879005, NDCG: 0.1029159, F1: 0.0770494, HIT_RATE: 0.4040454, ARHR_ALL_HITS: 0.2267537, NOVELTY: 0.0049504, AVERAGE_POPULARITY: 0.2091286, DIVERSITY_MEAN_INTER_LIST: 0.9765422, DIVERSITY_HERFINDAHL: 0.9976445, COVERAGE_ITEM: 0.2341149, COVERAGE_ITEM_HIT: 0.0596921, ITEMS_IN_GT: 0.7414086, COVERAGE_USER: 0

### Check the best model

In [19]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['result_on_test_best', 'hyperparameters_best', 'cutoff_to_optimize', 'time_on_validation_avg', 'hyperparameters_df', 'time_on_train_total', 'result_on_earlystopping_df', 'hyperparameters_best_index', 'time_on_validation_total', 'metric_to_optimize', 'time_on_last_df', 'result_on_validation_best', 'time_on_test_total', 'exception_list', 'time_on_train_avg', 'result_on_validation_df', 'time_on_test_avg', 'time_df', 'result_on_last', 'algorithm_name_search', 'result_on_test_df', 'algorithm_name_recommender'])

In [20]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

,topK,l2_norm
0,266,23.205627
1,200,200.0
2,227,56.530583
3,299,126.611602
4,224,138.487854
5,207,76.654026


In [21]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
0,10,0.060223,0.119292,0.106924,0.02762,0.055271,0.187901,0.102916,0.077049,0.404045,0.226754,...,0.770518,0.311324,0.770518,0.041058,9.923605,0.998009,0.120044,0.765337,1.844795,0.362719
1,10,0.058948,0.120074,0.108224,0.026794,0.054751,0.184772,0.101983,0.076324,0.401654,0.221639,...,0.770518,0.309482,0.770518,0.020941,8.800352,0.994168,0.061227,0.678708,2.765144,0.340754
2,10,0.061299,0.123271,0.110748,0.027919,0.056277,0.189734,0.104936,0.078918,0.413611,0.228916,...,0.770518,0.318695,0.770518,0.030063,9.448145,0.996896,0.087896,0.728668,2.208657,0.353111
3,10,0.059894,0.121838,0.109778,0.027313,0.055712,0.188099,0.103692,0.077503,0.406536,0.225805,...,0.770518,0.313244,0.770518,0.022564,8.945357,0.994848,0.065971,0.689892,2.654932,0.343118
4,10,0.060014,0.122108,0.110099,0.027334,0.055662,0.187707,0.103693,0.077683,0.406935,0.225621,...,0.770518,0.313551,0.770518,0.022948,8.973625,0.995036,0.067093,0.692072,2.617408,0.343858
5,10,0.0612,0.123625,0.111196,0.027833,0.056316,0.189786,0.10501,0.078948,0.412714,0.228676,...,0.770518,0.318004,0.770518,0.027777,9.326627,0.996568,0.081212,0.719296,2.296639,0.350724


In [22]:
result_best_on_test = search_metadata["result_on_last"]
result_best_on_test

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.091638,0.159614,0.135455,0.046868,0.081518,0.265553,0.146483,0.109319,0.510069,0.349573,...,0.800614,0.408369,0.800614,0.034213,9.671261,0.997478,0.098672,0.744987,2.006825,0.285766


In [23]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

{'topK': 227, 'l2_norm': 56.530583312788664}

In [12]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

In [13]:
old_recommender = EASE_R_Recommender(URM_train)
old_recommender.fit(topK=150,l2_norm=100)

EASE_R_Recommender: URM Detected 875 ( 6.7%) users with no interactions.
EASE_R_Recommender: URM Detected 446 ( 2.0%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 8.50 min


In [15]:
def precision(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    return precision_score

def recall(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score

def AP(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

def evaluate_model(URM_test, recommender_object, at=10):

    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_AP = 0.0

    num_eval = 0


    for user_id in range(URM_test.shape[0]):

        relevant_items = URM_test.indices[URM_test.indptr[user_id]:URM_test.indptr[user_id+1]]

        if len(relevant_items)>0:

            recommended_items = recommender_object.recommend(user_id, cutoff=at,remove_seen_flag=True)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)

    mean_precision = cumulative_precision / num_eval
    mean_recall = cumulative_recall / num_eval
    MAP = cumulative_AP / num_eval

    print("Recommender results are: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, MAP))
    return MAP, mean_precision, mean_recall

In [17]:
evaluate_model(URM_test,old_recommender)

Recommender results are: Precision = 795.8000, Recall = 1196.7690, MAP = 0.0639


(0.06389142073781028, 0.0760366902350513, 0.11434827439317237)

In [18]:
new_recommender = EASE_R_Recommender(URM_train)
new_recommender.fit(topK=227,l2_norm=56.530583)

EASE_R_Recommender: URM Detected 875 ( 6.7%) users with no interactions.
EASE_R_Recommender: URM Detected 446 ( 2.0%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 8.30 min


In [19]:
evaluate_model(URM_test,new_recommender)

Recommender results are: Precision = 803.5000, Recall = 1192.2011, MAP = 0.0642


(0.06421788398053065, 0.07677240588572974, 0.11391182095878476)

# Final model: train on whole dataset

In [21]:
recommender = EASE_R_Recommender(URM_all)
recommender.fit(topK=227,l2_norm=56.530583)

EASE_R_Recommender: URM Detected 387 ( 3.0%) users with no interactions.
EASE_R_Recommender: URM Detected 126 ( 0.6%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 8.32 min


## Predictions

In [25]:
user_id = 62
print(f'Predicting for user - {user_id}')
prediction = recommender.recommend(user_id, cutoff=10)
print(f"The prediction is {prediction}")

Predicting for user - 62
The prediction is [278, 411, 238, 85, 106, 950, 493, 67, 583, 291]


In [ ]:
pred_path = '/kaggle/input/recsyscourse/data_target_users_test.csv'

urm_pred_df = pd.read_csv(filepath_or_buffer=pred_path,
                                sep=",",
                                header=0,
                                dtype={0:int},
                                engine='python')

urm_pred_df.columns = ["UserID"]
len(urm_pred_df['UserID'])

---

# Submission

In [1]:
!pip install "numpy<1.24.0"

In [2]:
import numpy
print(numpy.__version__)

1.23.5


In [3]:
import sys
sys.path.append("RecSys_Course_AT_PoliMi")

In [5]:
import time
from matplotlib import pyplot
import pandas as pd
import numpy as np
import scipy.sparse as sps
import random as rnd

from scipy.sparse import *

urm_path = '/kaggle/input/recsyscourse/data_train.csv'
urm_all_df = pd.read_csv(filepath_or_buffer=urm_path,
                                sep=",",
                                header=0,
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

urm_all_df.columns = ["UserID", "ItemID", "Interaction"]

urm_all_df.head(10)

print ("The number of interactions is {}".format(len(urm_all_df)))

userID_unique = urm_all_df["UserID"].unique()
itemID_unique = urm_all_df["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(urm_all_df)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print ("Average interactions per user {:.2f}".format(n_interactions/n_users))
print ("Average interactions per item {:.2f}\n".format(n_interactions/n_items))

print ("Sparsity {:.2f} %".format((1-float(n_interactions)/(n_items*n_users))*100))

# Remove empty profiles

# Remove empty IDs
mapped_id, original_id = pd.factorize(urm_all_df["UserID"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(urm_all_df["ItemID"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)



# Let's create the mapping from items indices to original item_ids
item_index_to_original_ID = pd.Series(item_original_ID_to_index.index,index = item_original_ID_to_index.values)



original_item_ID = 125
print("New index for item {} is {}".format(original_item_ID, item_original_ID_to_index[original_item_ID]))


urm_all_df["UserID"] = urm_all_df["UserID"].map(user_original_ID_to_index)
urm_all_df["ItemID"] = urm_all_df["ItemID"].map(item_original_ID_to_index)
urm_all_df.head(n=10)

userID_unique = urm_all_df["UserID"].unique()
itemID_unique = urm_all_df["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(urm_all_df)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print ("Average interactions per user {:.2f}".format(n_interactions/n_users))
print ("Average interactions per item {:.2f}\n".format(n_interactions/n_items))

print ("Sparsity {:.2f} %".format((1-float(n_interactions)/(n_items*n_users))*100))

urm_all = sps.coo_matrix((urm_all_df["Interaction"].values,
                          (urm_all_df["UserID"].values, urm_all_df["ItemID"].values)))
urm_all

The number of interactions is 478730
Number of items	 22222, Number of users	 12638
Max ID items	 22347, Max Id users	 13024

Average interactions per user 37.88
Average interactions per item 21.54

Sparsity 99.83 %
New index for item 125 is 93
Number of items	 22222, Number of users	 12638
Max ID items	 22221, Max Id users	 12637

Average interactions per user 37.88
Average interactions per item 21.54

Sparsity 99.83 %


<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in COOrdinate format>

In [6]:
class EASERRecommender(object):

    def __init__(self, URM, model):
        self.URM = URM
        self.model = model

    def recommend(self, user_id, at=None, exclude_seen=True, users_not_in_train=[]):
        # Check if user_id is a valid index
        if user_id < 0 or user_id >= self.URM.shape[0]:
            print(f"Invalid user_id: {user_id}")
            return

        # Check if user_id not in train use the topRec
        if user_id in users_not_in_train:
            return ["517 189 44 0 284 808 285 1 557 1266"]

        # compute the scores using the fitted model
        scores = self.model._compute_score_W_dense(user_id)[0]

        if exclude_seen:
            scores = self.filter_seen(user_id, scores)

        # rank items
        ranking = scores.argsort()[::-1]

        return ranking[:at]


    def filter_seen(self, user_id, scores):

        start_pos = self.URM.indptr[user_id]
        end_pos = self.URM.indptr[user_id+1]

        user_profile = self.URM.indices[start_pos:end_pos]

        scores[user_profile] = -np.inf

        return scores

In [7]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

model = EASE_R_Recommender(urm_all)
model.fit(topK = 227,l2_norm = 56.530583)

EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 8.50 min


In [14]:
urm_all = urm_all.tocsr()

In [15]:
recommender_easer = EASERRecommender(urm_all,model)

#### Recommendations

In [10]:
pred_path = '/kaggle/input/recsyscourse/data_target_users_test.csv'

urm_pred_df = pd.read_csv(filepath_or_buffer=pred_path,
                                sep=",",
                                header=0,
                                dtype={0:int},
                                engine='python')

urm_pred_df.columns = ["UserID"]
len(urm_pred_df['UserID'])

10882

In [16]:
pred_df = pd.DataFrame(columns = ['user_id','item_list'])

for userid in urm_pred_df['UserID']:
  if userid in user_original_ID_to_index.keys():
    # Map user ID to the index of model user space
    usr_idx = user_original_ID_to_index[userid]
    recomendatoins = recommender_easer.recommend(usr_idx, at=10,exclude_seen=True)
    # Map item indices from model item space to original item ID
    recomendatoins = [item_index_to_original_ID[idx] for idx in recomendatoins]
  else:
    recomendatoins = ["517 189 44 0 284 808 285 1 557 1266"]

  recomendatoins = " ".join(str(item) for item in recomendatoins)
  pred_df.loc[len(pred_df)] = [userid,recomendatoins]

In [17]:
pred_df

,user_id,item_list
0,1,101 36 403 506 123 515 52 8 694 977
1,2,1095 12 47 50 102 28 1522 949 656 196
2,3,59 536 648 4252 4 259 584 956 857 2172
3,4,249 28 50 1 7 5 136 139 171 146
4,5,1570 170 77 131 95 1511 135 270 116 826
...,...,...
10877,13020,6450 6198 105 6452 7395 6749 7394 155 627 161
10878,13021,133 13 265 579 6451 6179 7027 6426 6720 139
10879,13022,1411 1668 809 1446 1674 1013 1258 776 118 314
10880,13023,706 329 1124 1107 1290 96 32 138 1534 208


In [20]:
pred_df.to_csv('./pred_easer_Bre.csv',index=False)

In [21]:
model.W_sparse

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00097784,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [23]:
from scipy.sparse import save_npz
save_npz("./easer.npz", csr_matrix(model.W_sparse) )